Data cleaning

The year was collected to be from 1923 to 2023.

We first would chose only the desired columns to work with before cleaning.
To clean it, first need to find what needs to be cleaned.

In [ ]:
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/MyDrive/Raw disaster dataset - Sheet1.csv', header=0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Preview the data
print(data.head(150))
print(data.columns)

In [ ]:
# Chosing to keep only some columns
columns = ['Year','Disaster Group', 'Disaster Subgroup',
       'Disaster Type', 'Disaster Subtype', 'Disaster Subsubtype', 'Country', 'ISO', 'Location',
       'Origin', 'Associated Dis', 'Associated Dis2', 'Dis Mag Value',
       'Dis Mag Scale', 'Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month',
       'End Day', 'Number of Days', 'Total Deaths', 'No Injured',
       'No Affected', 'No Homeless', 'Total Affected',
       'Reconstruction Costs (\'000 US$)',
       'Insured Damages, Adjusted (\'000 US$)',
       'Total Damages, Adjusted (\'000 US$)']

filtered = data[columns]
data = filtered

# See which column has any null
for col in columns:
  print(col, ":" , data[col].isnull().any())

Count how many missing for the important columns (total dmg adjusted, start month. start day, end yer, end month, end day)
--> Result: start month, start day, end month, end day all has missing values

As Affected is too vaguely defined, we'd skip the use of it completely (teh scale of hreporting this is inconsistent across areas), thus no reliable insights could be derived from them.

Some columns are allowed to be null, but some are not, dropping all null records in those that are not (total damage and start year can't be null)

Dropping na in 2 columns: Total Damages, Adjusted (\'000 US$), Start Month


---
*Do not keep location as it is to hard to deal with.*

---

Keep reconstruction cost, insured , no homeless, no injured, no death, associated disasters, dis value, dis mag scale, origin, diaaster group, subgroup, type, subtype, subsubtype, year.

Only keeping the adjusted ones.

Using Python to extract those data and handle missing data

In [ ]:
data.info()

In [ ]:
keep = ['Year','Disaster Group', 'Disaster Subgroup',
       'Disaster Type', 'Disaster Subtype', 'Disaster Subsubtype',
       'Origin', 'Associated Dis', 'Associated Dis2', 'Dis Mag Value',
       'Dis Mag Scale', 'Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month',
       'End Day', 'Number of Days', 'Total Deaths', 'No Injured',
       'No Homeless', 'Total Damages, Adjusted (\'000 US$)']

# condition - accpeted years after 1979 only
year_cond = data['Year'] > 1979

# condition - cleaning missing value data in days and month

data = data[keep]
data = data[year_cond]
data = data.dropna(subset = ['Start Month', 'Start Day', 'End Month', 'End Day',  'Total Damages, Adjusted (\'000 US$)'])

In [ ]:
data.info()

We'll only interested in analyzing Meteorological, Hydrological and Climatological data so we'll only keep those

In [ ]:
# condition on disaster sub-group
cond1 = ['Meteorological', 'Hydrological', 'Climatological']
data = data[data['Disaster Subgroup'].isin(cond1)]

When considering only those three subtypes, disaster magnitudes became excessively empty. We get a look at it and decide to remove it too.

In [ ]:
data['Dis Mag Value'].info()

There is only 189 non-null values out of 588. We can figure out how to fill it in, but for now the practice is to make it binary. (compare between yes there was measurement vs no there was not any measurement)

In [ ]:
# Making another column as the binary of the Dis Mag Value column

data['Dis Mag Value_binary'] = data['Dis Mag Value'].notnull().astype(int)

Other attributes, including total death, number injured, number homeless have many null values too. However, on the basis that not all disaster must have casualties, we would need to consider it more throroughly to decide on a handling procedure.

For now, preliminary cleaning process is done.

NEED FURTHER CLEANING OF DATA - TOO FEW LANDSLIDE AND EXTREME TEMPERATURE AND DROUGHT.

In [ ]:
data.groupby('Disaster Type').count()

In [ ]:
data = data[data['Disaster Type'].isin(['Storm', 'Flood', 'Wildfire'])]

We are technically done with cleaning the dataset. Now we would start doing some analysis and trasnform the data if need to.

In [ ]:
# As total damage is too skewed (left skewed) and having too many outliers, using natural log to rescale the shape
# Downside is now we are analyzing not on the total dmg adj but the log of it
data["Log of Total Damages, Adjusted ('000 US$)"] = np.log(data["Total Damages, Adjusted ('000 US$)"])

After the transformation, there are still outliers in the Storm category of Disaster Type.

--> ***keep*** the outliers, analyzing them on a one-to-one basis.


In [ ]:
# Many nulls in Associated Disaster 1 and 2. Thus make a new column - binary of the Associated disaster (1 for yes and 0 for no)

data['Asso_Dist1_Binary'] = data['Associated Dis'].notnull().astype(int)
data['Asso_Dist2_Binary'] = data['Associated Dis2'].notnull().astype(int)

# Sums up the 2 binary cols for asso_dist -> get the number of associated disasters
data['Number_of_asso_dist'] = data['Asso_Dist1_Binary'] + data['Asso_Dist2_Binary']


Total death, number injured and number homeless - hard to differentiate if there is a lack of records or the nulls are just 0, so best practice is to turn it into binary.

In [ ]:
# Making new columns as the binary of the total death, no injured and no homeless columns

data['Total_Deaths_binary'] = data['Total Deaths'].notnull().astype(int)
data['No_Injured_binary'] = data['No Injured'].notnull().astype(int)
data['No_Homeless_binary'] = data['No Homeless'].notnull().astype(int)

In [ ]:
data['No Injured'].info()
data['No Homeless'].info()
data.groupby('Disaster Type').count()

In [ ]:
data.to_csv('cleaned_data.csv', index = False)